### Question
Write out two equations: (1) the equation for a the linear model that predicts y from X, and
(2) the equation for computing the Residual Sum of Squares (RSS) for the linear model,
given data, vector x, and parameters, vector β.
* See equations 3.1 and 3.2 in the Elements of Statistical Learning book
* Feel free to ignore the intercept term for this homework (e.g. β0)

In [12]:

%%latex
\begin{equation}
    y =   \beta_0 + \sum_{j=1}^{p} X_j \beta_j
\end{equation}

<IPython.core.display.Latex object>

In [16]:

%%latex
\begin{equation}
    RSS(\beta) = \sum_{i=1}^{N}( y_i - \beta_0 - \sum_{j=1}^{p} x_{ij} \beta_j)
\end{equation}

<IPython.core.display.Latex object>

Translate these equations into into code in the form of two functions 
* The first function should compute the estimated value of y, y_hat, for particular values of x, and β. That is, there should be two arguments, one for the data and one for the linear function parameters.
* The second function should compute the RSS for the first function

### Ans:
the funcions RSS and calculate_fx are defined later

In [17]:
import pandas as pd
import scipy as sp
import numpy as np
from scipy.optimize import minimize
import time

Read the wine data set

In [18]:
df = pd.read_csv("./winequality-red.csv",sep=";")

Look through the data

In [19]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [20]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
fixed acidity           1599 non-null float64
volatile acidity        1599 non-null float64
citric acid             1599 non-null float64
residual sugar          1599 non-null float64
chlorides               1599 non-null float64
free sulfur dioxide     1599 non-null float64
total sulfur dioxide    1599 non-null float64
density                 1599 non-null float64
pH                      1599 non-null float64
sulphates               1599 non-null float64
alcohol                 1599 non-null float64
quality                 1599 non-null int64
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


#### intial conclusion
As there seem to be no null values and also as all the data are already numeric there is no need to do any preprocessing to perform linear regression

Next we split the data into train and test set

In [22]:
X,y = df.iloc[:,:-1],df.iloc[:,-1]

In [23]:
def train_test_split(X,y):
    np.random.seed(5)
    rows = np.random.rand(len(df)) < 0.8
    train_X = X[rows]
    test_X = X[~rows]  
    train_y = y[rows]
    test_y = y[~rows]
    return train_X,train_y,test_X,test_y


In [24]:
train_X,train_y,test_X,test_y = train_test_split(X,y)

initialize the beta array

In [25]:
np.random.seed(6)
beta = np.random.rand(len(train_X.columns)+1)

In [26]:
#function to calculate yhat
def calculate_fx(x,beta):
    f_x = beta[0]
    for i in range(len(x)):
        f_x = f_x + beta[i+1]*x[i]
    return f_x


In [27]:
#function to calculate RSS
def RSS(beta,X,y):
    rss = 0
    for i in range(len(X)):
        rss = rss + (y.iloc[i] - calculate_fx(X.iloc[i],beta))**2
    return rss

In [28]:
#train the predictor using minimize
sol = minimize(RSS,beta,(train_X,train_y),'SLSQP')


In [31]:
#calculate the train rss
RSS(sol.x,train_X,train_y)

535.6697948469422

In [32]:
# Calculate the test rss
RSS(sol.x,test_X,test_y)

134.04984779225526

In [29]:
# Calculate the test rss per row
RSS(sol.x,test_X,test_y)/len(test_X)

0.4242083790894154

In [30]:
#calculate the train rss per row
RSS(sol.x,train_X,train_y)/len(train_X)

0.4175134800054109

In [15]:
#max min normalization function
def norm_min_max_df(df):
    df_norm = (df-df.min())/(df.max()-df.min())
    return df_norm

In [16]:
#normalize the train data
train_X_norm = norm_min_max_df(train_X)

In [200]:
#minimize using the normalized data
sol_norm = minimize(RSS,beta,(train_X_norm,train_y),'SLSQP')

In [201]:
sol_norm.x

array([ 5.71707369,  0.66862169, -1.10560747, -0.05422717,  0.69295629,
       -1.21788643,  0.31816226, -1.01508182, -0.87283348, -0.22191063,
        1.58857632,  1.45771155])

In [17]:
#normalize the test data
test_X_norm = norm_min_max_df(test_X)

In [229]:
#calculate the train RSS for train norm data
RSS(sol_norm.x,train_X_norm,train_y)

535.6697947246332

In [204]:
#calculate the RSS for test norm data
RSS(sol_norm.x,test_X_norm,test_y)

141.5634852893919

In [208]:
print("bias is {}".format(sol_norm.x[0]))
for i in range(train_X.shape[1]):
    print("normalized beta values of {} is {} ".format(df.columns[i],sol_norm.x[i+1]))

bias is 5.717073694147023
normalized beta values of fixed acidity is 0.6686216890214317 
normalized beta values of volatile acidity is -1.1056074746397424 
normalized beta values of citric acid is -0.054227170904254975 
normalized beta values of residual sugar is 0.6929562878678145 
normalized beta values of chlorides is -1.2178864251688428 
normalized beta values of free sulfur dioxide is 0.31816226368500433 
normalized beta values of total sulfur dioxide is -1.0150818158427197 
normalized beta values of density is -0.8728334781086817 
normalized beta values of pH is -0.22191063328947874 
normalized beta values of sulphates is 1.5885763157956587 
normalized beta values of alcohol is 1.457711549158819 


### Question
What are the qualitative results from your model? Which features seem to be most
important? Do you think that the magnitude of the features in X may affect the
results (for example, the average total sulfur dioxide across all wines is 46.47, but
the average chlorides is only 0.087).

### Ans : 
As the features were not normalized before, comparing the beta values of the model would not be a good idea. The reason is the the beta values are affected by the scale of the data. The higher the magnitude the lower the beta values. For e.g. if the beta values for a quantity were in Kilometer and for the same quantity if they were measured in millimeter than we would get different beta values as 1 unit change in kilometer would have much more difference in the beta values. 

Also just looking at the beta values even on normalized data might not be a definite measure of effect on the dependant variable as this is affected by the correlation between the independant variables as well.

Just looking at the normalized beta values and assuming that there is no correlation between the variable we get , the top 5 most important features as alcohol,sulphates,chlorides,volatile acidity and total sulfur dioxide.

The magnitude of the data doesn't have a huge effect in simple linear regression though as the beta values are able to change based on the scale of the input

### Question
How well does your model fit? You should be able to measure the goodness of fit,
RSS, on both the training data and the test data, but only report the results on the
test data. In Machine Learning we almost always only care about how well the
model fits on data that has not been used to fit the model, because we need to use
the model in the future, not the past. Therefore, we only report performance with
holdout data, or test data

### Ans : 

Our model seems to be a good fit as the RSS value per example on average we get is pretty low.


### Varying the beta values

In [33]:
beta_sol_list = []

In [251]:
for i in range(5):
    print("running {} set of beta".format(i))
    beta_new = np.random.uniform(low=0.2, high=50, size=(train_X.shape[1]+1,))
    sol_beta = minimize(RSS,beta_new,(train_X,train_y),'SLSQP')
    rss = RSS(sol_beta.x,test_X,test_y)
    print("test rss for {} set of beta is {}".format(i,rss))
    beta_sol_list.append((beta_new,sol_beta,rss))
    

running 0 set of beta
test rss for 0 set of beta is 134.04975225744693
running 1 set of beta
test rss for 1 set of beta is 134.0473031013729
running 2 set of beta
test rss for 2 set of beta is 134.04771518629946
running 3 set of beta
test rss for 3 set of beta is 134.04846890993986
running 4 set of beta
test rss for 4 set of beta is 134.04873963225228


### Question

Does the end result or RSS change if you try different initial values of β? What
happens if you change the magnitude of the initial β?

### Ans
The end RSS might change as the different beta values may lead to the algorithm reaching different local minimas on the data set where they converge.

Changing the magnitude will have a similar effect on the data set. As is observed from our experiment. changing the beta values from our initial observation converged to a different places of RSS of 134.047 compared to 141.56

### Varying the solvers

In [177]:
solvers = [
    'BFGS' ,
    'COBYLA',
    'SLSQP'
]

In [178]:
sol_list_time = []

In [179]:
#minimize using different solvers
for i in range(len(solvers)):
    #minimize
    print("starting minimization using {}".format(solvers[i]))
    start = time.time()
    sol = minimize(RSS,beta,(train_X,train_y),solvers[i])
    end = time.time()
    print(end - start)
    sol_list.append((solvers[i],end-start,sol))

starting minimization using BFGS
400.64189171791077
starting minimization using COBYLA
275.44445300102234
starting minimization using SLSQP
97.33751463890076


In [218]:

for i in range(len(sol_list)):
    rss  = RSS(sol_list[i][2].x,test_X,test_y)
    print("the optimizer {} gave a rss of {} in {:0.2f} seconds".format(sol_list[i][0],rss,sol_list[i][1]))

the optimizer BFGS gave a rss of 134.04847983929395 in 400.64 seconds
the optimizer COBYLA gave a rss of 179.22404520100423 in 275.44 seconds
the optimizer SLSQP gave a rss of 134.04984779225526 in 97.34 seconds


### Question
Does the choice of solver method change the end result or RSS?

### Ans:
The choice of the solver definetly affects the end result as different solvers have different assumptions and this affects the way optimize the problem. The solver might also affect the runtime of the problem as is clear from our tests.

In our experiment with 3 solvers they converged to different location and the took different amounts of time

## Regularization

### Question 
Try adding in an L2 (aka Ridge) regularization penalty to your model above to create a new,
regularized model. See equation 3.41 for guidance. You will need to choose a value of
lambda, so start with something small, like 0.01. That is:
* lambda = 0.01
* y_hat_reg = y_hat + lambda * np.sum(beta**2)

### Ans:

The answer is coded below

In [21]:
def ridge_optimization(beta,X,y,lambda_val):
    rss  = RSS(beta,X,y)
    optim =  rss + lambda_val * np.sum(beta**2)
    return optim

In [222]:
lambda_val = 0.01

In [224]:
ridge_sol = minimize(ridge_optimization,beta,(train_X_norm,train_y,lambda_val),'SLSQP')

In [227]:
RSS(ridge_sol.x,train_X_norm,train_y)

535.6701586037094

In [228]:
RSS(ridge_sol.x,test_X_norm,test_y)

141.51815519259

### Question
How does RSS on the training data change? How does RSS on the test data change?

### Ans :
The RSS on the training data and the test data doesn't change much. This is mainly because the value of lambda that we took is very small

In [230]:
#trying to tune lambda between 0 and 1
lambda_arr = np.linspace(0,1,11)

In [231]:
lambda_arr

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [232]:
lambda_sol_list = []

In [235]:
for i in range(len(lambda_arr)):
    lambda_val = lambda_arr[i]
    print("running for lambda {}".format(lambda_val))
    ridge_sol_lambda = minimize(ridge_optimization,beta,(train_X_norm,train_y,lambda_val),'SLSQP')
    test_rss = RSS(ridge_sol_lambda.x,test_X_norm,test_y)
    print("test rss of {} for lambda {}".format(test_rss,lambda_val))
    lambda_sol_list.append((lambda_val,ridge_sol_lambda,test_rss))

running for lambda 0.0
test rss of 141.5634852893919 for lambda 0.0
running for lambda 0.1
test rss of 141.13816399445025 for lambda 0.1
running for lambda 0.2
test rss of 140.76755482747143 for lambda 0.2
running for lambda 0.30000000000000004
test rss of 140.44527986087334 for lambda 0.30000000000000004
running for lambda 0.4
test rss of 140.166328580241 for lambda 0.4
running for lambda 0.5
test rss of 139.926248025708 for lambda 0.5
running for lambda 0.6000000000000001
test rss of 139.72121667065397 for lambda 0.6000000000000001
running for lambda 0.7000000000000001
test rss of 139.547804145505 for lambda 0.7000000000000001
running for lambda 0.8
test rss of 139.4028841152773 for lambda 0.8
running for lambda 0.9
test rss of 139.2838467092026 for lambda 0.9
running for lambda 1.0
test rss of 139.18817864973707 for lambda 1.0


In [19]:
#trying to tune lambda between 1 and 11
lambda_arr = np.linspace(1,11,11)
lambda_sol_list_1_11 = []

In [22]:
for i in range(len(lambda_arr)):
    lambda_val = lambda_arr[i]
    print("running for lambda {}".format(lambda_val))
    ridge_sol_lambda = minimize(ridge_optimization,beta,(train_X_norm,train_y,lambda_val),'SLSQP')
    test_rss = RSS(ridge_sol_lambda.x,test_X_norm,test_y)
    print("test rss of {} for lambda {}".format(test_rss,lambda_val))
    lambda_sol_list_1_11.append((lambda_val,ridge_sol_lambda,test_rss))

running for lambda 1.0
test rss of 139.18817864973707 for lambda 1.0
running for lambda 2.0
test rss of 139.13904140294122 for lambda 2.0
running for lambda 3.0
test rss of 140.0072302590598 for lambda 3.0
running for lambda 4.0
test rss of 141.25529883006897 for lambda 4.0
running for lambda 5.0
test rss of 142.65035210402706 for lambda 5.0
running for lambda 6.0
test rss of 144.08419967845535 for lambda 6.0
running for lambda 7.0
test rss of 145.5049529399709 for lambda 7.0
running for lambda 8.0
test rss of 146.8887796490854 for lambda 8.0
running for lambda 9.0
test rss of 148.22427615317645 for lambda 9.0
running for lambda 10.0
test rss of 149.5089611159303 for lambda 10.0
running for lambda 11.0
test rss of 150.74034319575995 for lambda 11.0


In [23]:
#trying to tune lambda between 1 and 2
lambda_arr = np.linspace(1,2,11)
lambda_sol_list_1_2= []
for i in range(len(lambda_arr)):
    lambda_val = lambda_arr[i]
    print("running for lambda {}".format(lambda_val))
    ridge_sol_lambda = minimize(ridge_optimization,beta,(train_X_norm,train_y,lambda_val),'SLSQP')
    test_rss = RSS(ridge_sol_lambda.x,test_X_norm,test_y)
    print("test rss of {} for lambda {}".format(test_rss,lambda_val))
    lambda_sol_list_1_2.append((lambda_val,ridge_sol_lambda,test_rss))

running for lambda 1.0
test rss of 139.18817864973707 for lambda 1.0
running for lambda 1.1
test rss of 139.1137929181558 for lambda 1.1
running for lambda 1.2
test rss of 139.0588192130597 for lambda 1.2
running for lambda 1.3
test rss of 139.02142758236263 for lambda 1.3
running for lambda 1.4
test rss of 139.00004271387738 for lambda 1.4
running for lambda 1.5
test rss of 138.99339225659574 for lambda 1.5
running for lambda 1.6
test rss of 139.00006285288785 for lambda 1.6
running for lambda 1.7000000000000002
test rss of 139.01874143485807 for lambda 1.7000000000000002
running for lambda 1.8
test rss of 139.048907680468 for lambda 1.8
running for lambda 1.9
test rss of 139.08925795264125 for lambda 1.9
running for lambda 2.0
test rss of 139.13904140294122 for lambda 2.0


### Question
What happens if you try different values of lambda? Can you roughly tune lambda to get
the best results on the test data?

### Ans:
Different values of lambda cause different test rss as the values of lambda has an effect on how high or low the calues of the beta vector can go. 

In our case we got the best test RSS ar lambda = 1.5

In [10]:
def lasso_optimization(beta,X,y,lambda_val):
    rss  = RSS(beta,X,y)
    optim =  rss + lambda_val * np.sum(np.absolute(beta))
    return optim

In [253]:
#trying to tune lambda between 0 and 10
lambda_arr = np.linspace(0,10,11)
lasso_lambda_sol_list = []

In [254]:
for i in range(len(lambda_arr)):
    lambda_val = lambda_arr[i]
    print("running for lambda {}".format(lambda_val))
    lasso_sol_lambda = minimize(lasso_optimization,beta,(train_X_norm,train_y,lambda_val),'SLSQP')
    test_rss = RSS(lasso_sol_lambda.x,test_X_norm,test_y)
    print("test rss of {} for lambda {}".format(test_rss,lambda_val))
    lasso_lambda_sol_list.append((lambda_val,lasso_sol_lambda,test_rss))

running for lambda 0.0
test rss of 141.5634852893919 for lambda 0.0
running for lambda 1.0
test rss of 139.74931077224946 for lambda 1.0
running for lambda 2.0
test rss of 138.17369422873608 for lambda 2.0
running for lambda 3.0
test rss of 137.1461399974861 for lambda 3.0
running for lambda 4.0
test rss of 136.6799777097008 for lambda 4.0
running for lambda 5.0
test rss of 136.7751208120813 for lambda 5.0
running for lambda 6.0
test rss of 137.9478110535365 for lambda 6.0
running for lambda 7.0
test rss of 138.81337907591174 for lambda 7.0
running for lambda 8.0
test rss of 139.26013296702493 for lambda 8.0
running for lambda 9.0
test rss of 139.8939864055285 for lambda 9.0
running for lambda 10.0
test rss of 140.536491253383 for lambda 10.0


In [18]:
#trying to tune lambda between 4 and 5
lambda_arr = np.linspace(4,5,11)
lasso_lambda_sol_list_4_5 = []
for i in range(len(lambda_arr)):
    lambda_val = lambda_arr[i]
    print("running for lambda {}".format(lambda_val))
    lasso_sol_lambda = minimize(lasso_optimization,beta,(train_X_norm,train_y,lambda_val),'SLSQP')
    test_rss = RSS(lasso_sol_lambda.x,test_X_norm,test_y)
    print("test rss of {} for lambda {}".format(test_rss,lambda_val))
    lasso_lambda_sol_list_4_5.append((lambda_val,lasso_sol_lambda,test_rss))

running for lambda 4.0
test rss of 136.6799777097008 for lambda 4.0
running for lambda 4.1
test rss of 136.6632497926084 for lambda 4.1
running for lambda 4.2
test rss of 136.6534196102417 for lambda 4.2
running for lambda 4.3
test rss of 136.65055951808654 for lambda 4.3
running for lambda 4.4
test rss of 136.6505957555164 for lambda 4.4
running for lambda 4.5
test rss of 136.65651435139122 for lambda 4.5
running for lambda 4.6
test rss of 136.66919588234802 for lambda 4.6
running for lambda 4.7
test rss of 136.3177149579503 for lambda 4.7
running for lambda 4.8
test rss of 136.71161906013145 for lambda 4.8
running for lambda 4.9
test rss of 136.74035458320213 for lambda 4.9
running for lambda 5.0
test rss of 136.7751208120813 for lambda 5.0


### Question
Now try using an L1 (aka Lasso) regularization penalty instead. See equation 3.51 for
example. Report your findings on how RSS changes, and if you can roughly tune lambda.

### Ans:

We received even lower values of RSS with Lasso. Roughly tuning lasso we get the best test RSS of 136.31 for a lambda value of 4.7

### Question 
Again, do you think that the magnitude of the features in X may affect the results with
regularization?

### Ans:
Yes the magnitude of features of X has a very clear affect on the result in this case as the magnitudes of features affect the values of the beta values as explained before. In simple linear regression this was not a problem as the beta values were not constrained and could adjust to fit the scale of the input. But in this case as the beta is constrained, the input need to be normalized so that the scale doesn't affect the result